In [22]:
import pandas as pd

from src.all_in_one import *

# UNIQUE SCREENING BENEFICIARIES RECORDS

In [23]:
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr = get_df(scr_file.path, scr_file.sheet, config_file)
scr = scr.sort_values(by=['sc_s1'])
unique_scr = scr.drop_duplicates(subset='rid')

In [3]:
# unique_scr

# UNIQUE INTAKE BENEFICIARIES RECORD

In [24]:
intake_s2 = ["rid", "fcid", "service","nt_s2", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]
intake_s3 = ["rid", "fcid", "service","nt_s3", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]
intake_re = ["rid", "fcid", "service","nt_re", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]
int_criteria = ["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]
int_criteria0 = ["family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]



In [25]:
int_file = path_manager.get_data_file(Category.PS, PSFile.PSNT)
intake = get_df(int_file.path, int_file.sheet, config_file)
intake.insert(4, "service", None)
intake = intake.sort_values(by=['nt_s1', 'nt_s2', 'nt_s3', 'nt_re'], ascending=True)
unique_intake = intake.drop_duplicates(subset='rid', keep='last')

In [6]:
# unique_intake

In [7]:
# path = os.path.join(unvfvt23, 'ptreintBeneficiaries23.csv')
# ptreint = pd.read_csv(path)
# ptreint.insert(4, 'session_num', 0)
# ptreint = pd.merge(ptreint, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ptreint = pd.merge(ptreint, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ptreint

In [8]:
# ptreint.to_csv(path, index=False)

## PSYCHOSOCIAL INTAKE ASSESSMENT PROCESSING

In [9]:
intake_24 = filter_dataframe_on_date(intake, 'nt_s2', START, END)
intake_24 = intake_24[intake_s2]
intake_24["service"] = "int"
intake_24.rename(columns={"nt_s2": "date"}, inplace=True)

In [10]:
# intake_24

In [11]:
intake3_24 = filter_dataframe_on_date(intake, 'nt_s3', START, END)
intake3_24 = intake3_24[intake_s3]
intake3_24["service"] = "int"
intake3_24.rename(columns={"nt_s3": "date"}, inplace=True)

In [12]:
# intake3_24

## PSYCHOSOCIAL RE-INTAKE ASSESSMENT PROCESSING

In [13]:
reintake_24 = filter_dataframe_on_date(intake, 'nt_re', START, END)
reintake_24 = reintake_24[intake_re]
reintake_24["service"] = "re"
reintake_24.rename(columns={"nt_re": "date"}, inplace=True)

In [14]:
# reintake_24

In [15]:
int24 = pd.concat([intake_24, intake3_24, reintake_24])
int24 = pd.merge(int24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [17]:
int24.to_csv(r"../data/processed/hsh_report/intake324.csv", index=False)

In [5]:
# intake_test = pd.read_csv(r"../data/processed/hsh_report/intake324.csv")
# intake_test

In [ ]:
# _int24 = int24[int24['service'] == 'int']
# reint24 = int24[int24['service'] == 're']
# _int24_path= os.path.join(root, data, processed, "unvfvt24", "intBeneficiaries24.csv")
# reint24_path= os.path.join(root, data, processed, "unvfvt24", "reintBeneficiaries24.csv")
# _int24.to_csv(_int24_path, index=False)
# reint24.to_csv(reint24_path, index=False)

## GROUP PSYCHOSOCIAL THERAPY COUNSELING PROCESSING

In [20]:
gc_file = path_manager.get_data_file(Category.PS, PSFile.PSG)
gc_dataset = Dataset(config_file, gc_file.sheet)
gc = get_df(gc_file.path, gc_file.sheet, config_file)
gc_24 = filter_dataframe_on_date(gc, gc_dataset.dvars, START, END)
# gc_24 = gc_24[gc_24['gc_total'] >= 7]  # To be reconsidered 
gc_24_latestdate = gc_24[gc_dataset.dvars].max(axis=1)
gc_24.insert(4, 'service', 'gc')
gc_24.insert(5, 'date', gc_24_latestdate)

In [24]:
gc_24 = gc_24[['rid', 'fcid', 'service', 'date']]
gc_24 = pd.merge(gc_24, unique_intake[int_criteria], on='rid', how='left')
gc_24 = pd.merge(gc_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [26]:
# gc_24

In [27]:
gc_24.to_csv(r"../data/processed/hsh_report/gc324.csv", index=False)

## INDIVIDUAL PSYCHOSOCIAL THERAPY COUNSELING PROCESSING

In [29]:
# ic = get_df(ic_path, ic_sheet, config_file)
ic_files = path_manager.get_data_file(Category.PS, PSFile.PSI)
ic_dataset = Dataset(config_file, ic_files.sheet)
ic = get_df(ic_files.path, ic_files.sheet, config_file)
ic_24 = filter_dataframe_on_date(ic, ic_dataset.dvars, START, END)
ic_24_latestdate = ic_24[ic_dataset.dvars].max(axis=1)
ic_24.insert(4, "service", "ic")
ic_24.insert(5, "date", ic_24_latestdate)
ic_24 = ic_24[['rid', 'fcid', 'service', 'date']]
# ic_24.rename(columns={'ic_total': 'session_num'}, inplace=True)
ic_24 = pd.merge(ic_24, unique_intake[int_criteria], on='rid', how='left')
ic_24 = pd.merge(ic_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [31]:
# ic_24

In [32]:
ic_24.to_csv(r"../data/processed/hsh_report/ic324.csv", index=False)

## PSYCHOSOCIAL FOLLOW-UP ASSESSMENT PROCESSING

In [34]:
psfua_files = path_manager.get_data_file(Category.PS, PSFile.PSFU)
psfua_dataset = Dataset(config_file, psfua_files.sheet)
psfua = get_df(psfua_files.path, psfua_files.sheet, config_file)

In [36]:
psfua3_24 = filter_dataframe_on_date(psfua, 'fu_3m', START, END)
psfua3_24.insert(4, "service", "psfua3")
psfua3_24 = psfua3_24[['rid', 'fcid', 'service', 'fu_3m']].copy()
psfua3_24.rename(columns={'fu_3m': 'date'}, inplace=True)

In [38]:
psfua6_24 = filter_dataframe_on_date(psfua, 'fu_6m', START, END)
psfua6_24.insert(4, "service", "psfua6")
psfua6_24 = psfua6_24[['rid', 'fcid', 'service', 'fu_6m']].copy()
psfua6_24.rename(columns={'fu_6m': 'date'}, inplace=True)

In [40]:
psfua12_24 = filter_dataframe_on_date(psfua, 'fu_12m', START, END)
psfua12_24.insert(4, "service", "psfua12")
psfua12_24 = psfua12_24[['rid', 'fcid', 'service', 'fu_12m']].copy()
psfua12_24.rename(columns={'fu_12m': 'date'}, inplace=True)

In [ ]:
psfua3_24 = pd.merge(psfua3_24, unique_intake[int_criteria], on='rid', how='left')
psfua3_24 = pd.merge(psfua3_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

psfua6_24 = pd.merge(psfua6_24, unique_intake[int_criteria], on='rid', how='left')
psfua6_24 = pd.merge(psfua6_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

psfua12_24 = pd.merge(psfua12_24, unique_intake[int_criteria], on='rid', how='left')
psfua12_24 = pd.merge(psfua12_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [42]:
psfua3_24.to_csv(r"../data/processed/hsh_report/psfua324.csv", index=False)
psfua6_24.to_csv(r"../data/processed/hsh_report/psfua624.csv", index=False)
psfua12_24.to_csv(r"../data/processed/hsh_report/psfua1224.csv", index=False)

## THERAPUTIC DOCUMENTATION PROCESSING

In [6]:
td_file = path_manager.get_data_file(Category.PS, PSFile.TD)
td_dataset = Dataset(config_file, td_file.sheet)
td = get_df(td_file.path, td_file.sheet, config_file)

td_24 = filter_dataframe_on_date(td, td_dataset.dvars, START, END)
td_24_latestdate = td_24[td_dataset.dvars].max(axis=1)
td_24.insert(4, "service", "td")
td_24.insert(5, "date", td_24_latestdate)
td_24 = td_24[['rid', 'fcid', 'service', 'date']]
td_24 = pd.merge(td_24, unique_intake[int_criteria], on='rid', how='left')
td_24 = pd.merge(td_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [8]:
td_24.to_csv(r"../data/processed/hsh_report/td324.csv", index=False)

### Creative Documentation Workshop
Next time, TD should include all activities related to TD, where the output is one column contains value 1 if a beneficiary has participated in any activity of TD during the report period.

In [18]:
cws_file = path_manager.get_data_file(Category.PS, PSFile.CWS)
cws_dataset = Dataset(config_file, cws_file.sheet)
cws = get_df(cws_file.path, cws_file.sheet, config_file)

cws_24 = filter_dataframe_on_date(cws, cws_dataset.dvars, START, END)
cws_24_latestdate = cws_24[cws_dataset.dvars].max(axis=1)
cws_24.insert(4, "service", "cdw")
cws_24.insert(5, "date", cws_24_latestdate)
cws_24 = cws_24[['rid', 'fcid', 'service', 'date']]
cws_24 = pd.merge(cws_24, unique_intake[int_criteria], on='rid', how='left')
cws_24 = pd.merge(cws_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [20]:
cws_24.to_csv(r"../data/processed/hsh_report/cws324.csv", index=False)

## Trauma Resilience Workshop PROCESSING

In [ ]:
# trw = get_df(trw_path, trw_sheet, config_file)

In [21]:
trw_file = path_manager.get_data_file(Category.PS, PSFile.TRW)
trw_dataset = Dataset(config_file, trw_file.sheet)
trw = get_df(trw_file.path, trw_file.sheet, config_file)

trw_24 = filter_dataframe_on_date(trw, trw_dataset.dvars, START, END)
trw_24_latestdate = trw_24[trw_dataset.dvars].max(axis=1)
trw_24.insert(4, "service", "trw")
trw_24.insert(5, "date", trw_24_latestdate)
trw_24 = trw_24[['rid', 'fcid', 'service', 'date']]
trw_24 = pd.merge(trw_24, unique_intake[int_criteria], on='rid', how='left')
trw_24 = pd.merge(trw_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [23]:
trw_24.to_csv(r"../data/processed/hsh_report/trw324.csv", index=False)

## POST-EARTHQUAKE INTERVENTION PROCESSING

In [27]:
pei_file = path_manager.get_data_file(Category.PS, PSFile.PEI)
pei_dataset = Dataset(config_file, pei_file.sheet)
pei = get_df(pei_file.path, pei_file.sheet, config_file)

pei_24 = filter_dataframe_on_date(pei, pei_dataset.dvars, START, END)
pei_24_latestdate = pei_24[pei_dataset.dvars].max(axis=1)
pei_24.insert(4, "service", "pei")
pei_24.insert(5, "date", pei_24_latestdate)
pei_24 = pei_24[['rid', 'fcid', 'service', 'date']]
pei_24 = pd.merge(pei_24, unique_intake[int_criteria], on='rid', how='left')
pei_24 = pd.merge(pei_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [28]:
pei_24.to_csv(r"../data/processed/hsh_report/pei324.csv", index=False)

## GROUP PHYSIOTHERAPY PROCESSING

In [5]:
ptg_file = path_manager.get_data_file(Category.PT, PTFile.PTG)
ptg_dataset = Dataset(config_file, ptg_file.sheet)
ptg = get_df(ptg_file.path, ptg_file.sheet, config_file)

ptg_24 = filter_dataframe_on_date(ptg, ptg_dataset.dvars, START, END)
ptg_24_latestdate = ptg_24[ptg_dataset.dvars].max(axis=1)
# ptg_24 = ptg_24.copy()
# non_nat_counts = ptg_24[ptg_dataset.dvars].notna().sum(axis=1)
# ptg_24['pt_g_total'] = np.where(ptg_24['pt_g_total'] == 0, non_nat_counts, ptg_24['pt_g_total'])
# ptg_24 = ptg_24[ptg_24['pt_status'] != 'Drop Out']
ptg_24.insert(4, "service", "gpt")
ptg_24.insert(5, "date", ptg_24_latestdate)
ptg_24 = ptg_24[['rid', 'fcid', 'service', 'date']]

ptg_24 = pd.merge(ptg_24, unique_intake[int_criteria], on='rid', how='left')
ptg_24 = pd.merge(ptg_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [6]:
ptg_24.to_csv(r"../data/processed/hsh_report/ptg324.csv", index=False)

## INDIVIDUAL PHYSIOTHERAPY PROCESSING

In [8]:
pti_file = path_manager.get_data_file(Category.PT, PTFile.PTI)
pti_dataset = Dataset(config_file, pti_file.sheet)
pti = get_df(pti_file.path, pti_file.sheet, config_file)
# non_nat_counts = ipt[ipt_dataset.dvars].notna().sum(axis=1)
# ipt['pt_i_total'] = np.where(ipt['pt_i_total'] == 0, non_nat_counts, ipt['pt_i_total'])
pti_24 = filter_dataframe_on_date(pti, pti_dataset.dvars, START, END)
pti_24_latestdate = pti_24[pti_dataset.dvars].max(axis=1)
# pti_24 = pti_24.copy()
# non_nat_counts = pti_24[pti_dataset.dvars].notna().sum(axis=1)
# pti_24['pt_i_total'] = np.where(pti_24['pt_i_total'] == 0, non_nat_counts, pti_24['pt_i_total'])
pti_24.insert(4, "service", "ipt")
pti_24.insert(5, "date", pti_24_latestdate)
pti_24 = pti_24[['rid', 'fcid', 'service', 'date']]

pti_24 = pd.merge(pti_24, unique_intake[int_criteria], on='rid', how='left')
pti_24 = pd.merge(pti_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [10]:
pti_24.to_csv(r"../data/processed/hsh_report/pti324.csv", index=False)

## PHYSIOTHERAPY INTAKE ASSESSMENT PROCESSING

In [15]:
ptint_file = path_manager.get_data_file(Category.PT, PTFile.PTNT)
ptint_dataset = Dataset(config_file, ptint_file.sheet)
ptint = get_df(ptint_file.path, ptint_file.sheet, config_file)

ptint_24 = filter_dataframe_on_date(ptint, ptint_dataset.dvars, START, END)
ptint_24_latestdate = ptint_24[['pt_nt_s1', 'pt_nt_s2', 'pt_nt_re']].max(axis=1)
# _ptint23 = ptint23.copy()
ptint_24.insert(4, 'service', 'ptint')
ptint_24.insert(5, 'date', ptint_24_latestdate)
ptint_24 = ptint_24[['rid', 'fcid', 'service', 'date']]

ptint_24 = pd.merge(ptint_24, unique_intake[int_criteria], on='rid', how='left')
ptint_24 = pd.merge(ptint_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [19]:
ptint_24.to_csv(r"../data/processed/hsh_report/ptint324.csv", index=False)

## PHYSIOTHERAPY FOLLOW-UP ASSESSMENT PROCESSING

In [20]:
ptfua_file = path_manager.get_data_file(Category.PT, PTFile.PTFU)
ptfua_dataset = Dataset(config_file, ptfua_file.sheet)
ptfua = get_df(ptfua_file.path, ptfua_file.sheet, config_file)

In [32]:
ptfua3_24 = filter_dataframe_on_date(ptfua, 'fu_3m', START, END)
ptfua3_24.insert(4, "service", "ptfua3")
ptfua3_24 = ptfua3_24[['rid', 'fcid', 'service', 'fu_3m']]
ptfua3_24 = ptfua3_24.rename(columns={'fu_3m': 'date'})

In [22]:
ptfua6_24 = filter_dataframe_on_date(ptfua, 'fu_6m', START, END)
ptfua6_24.insert(4, "service", "ptfua6")
ptfua6_24 = ptfua6_24[['rid', 'fcid', 'service', 'fu_6m']]
ptfua6_24 = ptfua6_24.rename(columns={'fu_6m': 'date'})

In [23]:
ptfua12_24 = filter_dataframe_on_date(ptfua, 'fu_12m', START, END)
ptfua12_24.insert(4, "service", "ptfua12")
ptfua12_24 = ptfua12_24[['rid', 'fcid', 'service', 'fu_12m']]
ptfua12_24 = ptfua12_24.rename(columns={'fu_12m': 'date'})

In [35]:
ptfua3_24 = pd.merge(ptfua3_24, unique_intake[int_criteria], on='rid', how='left')
ptfua3_24 = pd.merge(ptfua3_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

ptfua6_24 = pd.merge(ptfua6_24, unique_intake[int_criteria], on='rid', how='left')
ptfua6_24 = pd.merge(ptfua6_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

ptfua12_24 = pd.merge(ptfua12_24, unique_intake[int_criteria], on='rid', how='left')
ptfua12_24 = pd.merge(ptfua12_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [36]:
ptfua3_24.to_csv(r"../data/processed/hsh_report/ptfua324.csv", index=False)
ptfua6_24.to_csv(r"../data/processed/hsh_report/ptfua624.csv", index=False)
ptfua12_24.to_csv(r"../data/processed/hsh_report/ptfua1224.csv", index=False)

In [40]:
# pd.read_csv(r"../data/processed/hsh_report/ptfua1224.csv")

## Case Management

In [31]:
cm = pd.read_excel(r"../data/raw/0924/other_services/TS_FC_meetingOutreachReferralFollowUpAdvocacy.xlsx", sheet_name=None)

referral = cm["IngoingOutgoingInternalReferral"]
reffu = cm["Follow-up"]
adv = cm["Advocacy"]

referral.to_csv(r"../data/processed/0924/16_ref_0924.csv", index=False)
reffu.to_csv(r"../data/processed/0924/17_reffu_0924.csv", index=False)
adv.to_csv(r"../data/processed/0924/18_adv_0924.csv", index=False)

In [33]:
# adv

In [34]:
ref = pd.read_csv(r"../data/processed/0924/16_ref_0924.csv", dtype={"FC_ID": "Int64"})
ref = ref[ref["OutgoingInternalIngoing"] == "outgoing"]

In [35]:
ref_24 = filter_dataframe_on_date(ref, "referralDate", START, END)
ref_24.insert(4, "service", "ref")
ref_24 = ref_24.rename(columns={"referral_ID": "rid"})
ref_24 = ref_24.rename(columns={"FC_ID": "fcid"})
ref_24 = ref_24.rename(columns={"referralDate": "date"})
ref_24 = ref_24[['rid', 'fcid', 'service', 'date']]
# ref_24["fcid"] = ref_24["fcid"].astype("Int64")

In [36]:
ref_24 = pd.merge(ref_24, unique_intake[int_criteria], on='rid', how='left')
ref_24 = pd.merge(ref_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
ref_24.to_csv(r"../data/processed/hsh_report/ref324.csv", index=False)

In [42]:
# pd.read_csv(r"../data/processed/hsh_report/ref324.csv", dtype={"fcid": "Int64"})

### Advocacy

In [43]:
adv = pd.read_csv(r"../data/processed/0924/18_adv_0924.csv", dtype={"FC_ID": "Int64"})

In [46]:
adv_24 = filter_dataframe_on_date(adv, "eventDate", START, END)
adv_24.insert(4, "service", "adv")
adv_24 = adv_24.rename(columns={"referral_ID": "rid"})
adv_24 = adv_24.rename(columns={"FC_ID": "fcid"})
adv_24 = adv_24.rename(columns={"eventDate": "date"})
adv_24 = adv_24[['rid', 'fcid', 'service', 'date']]

In [48]:
adv_24 = pd.merge(adv_24, unique_intake[int_criteria], on='rid', how='left')
adv_24 = pd.merge(adv_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [50]:
adv_24.to_csv(r"../data/processed/hsh_report/adv324.csv", index=False)

### TD IU Delivery, Communication, Advocacy Workshop

In [ ]:
# td_del = pd.read_excel('td_tracker.xlsx')
# td_del = td_del[['Ref #', 'FC ID #', 'IU Del', 'Com Pub', 'Adv Partic']]
# td_del = td_del.rename(columns={'Ref #': 'rid', 'FC ID #': 'fcid', 'IU Del': 'iu_del', 'Com Pub': 'com_pub', 'Adv Partic': 'adv_par'})

In [ ]:
# td_del_24 = filter_dataframe_on_date(td_del, ['iu_del', 'com_pub', 'adv_par'], start, end)
# td_del_24_latestdate = td_del_24[['iu_del', 'com_pub', 'adv_par']].max(axis=1)
# td_del_24.insert(4, "service", "td_del")
# td_del_24.insert(5, 'date', td_del_24_latestdate)
# td_del_24 = td_del_24[['rid', 'fcid', 'service', 'date']]

In [ ]:
# td_del_24 = pd.merge(td_del_24, unique_intake[int_criteria], on='rid', how='left')
# td_del_24 = pd.merge(td_del_24, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# td_del_24_path = os.path.join(root, data, processed, "unvfvt24", "tdDelBeneficiaries24.csv")
# td_del_24.to_csv(td_del_24_path, index=False)

## CONCATENATION

In [1]:
from src.dataset_aggregator import *

In [ ]:
files = r"../data/processed/hsh_report"
result_df = read_and_concatenate_datasets(
    files,
    file_type='auto',
    recursive='False',
    file_pattern='*.csv',
    read_kwargs={'encoding': 'utf8'},
    error_handling='skip'
)

In [4]:
hsh_report = result_df['data']
hsh_report['fcid'] = hsh_report['fcid'].astype('Int64')
hsh_report.to_csv(r"../data/processed/hsh_report/hsh_report324.csv", index=False)

In [5]:
hsh_report

,rid,fcid,service,date,family_of_missing,survivor_of_detention,primary_torture,secondary_torture,sexual_violation_a,sexual_violation_p,hrd,journalist,wov,stgbv,lgbti,other,sex,age
0,R1471,<NA>,adv,2024-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R0588,8110100010,adv,2024-08-26,Yes,No,No,Yes,No,No,No,No,Yes,No,No,No,Female,37.0
2,R0062,8110020048,adv,2024-08-26,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,No,No,Female,48.0
3,R0092,8110050011,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,46.0
4,R0063,8110050010,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,R0675,<NA>,trw,2024-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,23.0
314,R1474,<NA>,trw,2024-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,47.0
315,R0440,<NA>,trw,2024-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,62.0
316,R1021,<NA>,trw,2024-09-27,Yes,Yes,Yes,Yes,No,No,No,No,Yes,No,No,No,Male,39.0


In [6]:
hsh_report['service'].value_counts()

service
ref        67
gc         27
gpt        26
psfua6     25
ptfua6     24
psfua3     24
ptfua12    17
pei        17
ptfua3     17
cdw        16
adv        15
psfua12    15
trw        10
ipt         5
re          4
td          4
int         2
ptint       2
ic          1
Name: count, dtype: int64

In [19]:
hsh_rep_beneficiaries = hsh_report.dropna(subset=['fcid'])
uniq_hsh_report = hsh_rep_beneficiaries.drop_duplicates(subset=['rid'])
uniq_hsh_report = uniq_hsh_report[['rid', 'fcid', 'service', 'date']]
uniq_hsh_report

,rid,fcid,service,date
1,R0588,8110100010,adv,2024-08-26
2,R0062,8110020048,adv,2024-08-26
3,R0092,8110050011,adv,2024-08-26
4,R0063,8110050010,adv,2024-08-26
5,R0113,8110020060,adv,2024-08-26
...,...,...,...,...
300,R1474,8110060012,ref,2024-09-11
302,R0089,8110060012,ref,2024-09-18
304,R0393,8110050086,td,2024-08-13
306,R0582,8110050094,td,2024-07-31


In [20]:
# uniq_hsh_report[uniq_hsh_report[int_criteria0].isnull().all(axis=1)]

In [18]:
intake_dataset = pd.read_csv(r"../data/processed/0924/02_psint_0924.csv")

In [28]:
uniq_hsh_report_reb = pd.merge(uniq_hsh_report, unique_intake[int_criteria], on='rid', how='left')
uniq_hsh_report_reb = pd.merge(uniq_hsh_report_reb, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [29]:
uniq_hsh_report_reb

,rid,fcid,service,date,family_of_missing,survivor_of_detention,primary_torture,secondary_torture,sexual_violation_a,sexual_violation_p,hrd,journalist,wov,stgbv,lgbti,other,sex,age
0,R0588,8110100010,adv,2024-08-26,Yes,No,No,Yes,No,No,No,No,Yes,No,No,No,Female,37
1,R0062,8110020048,adv,2024-08-26,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,No,No,Female,48
2,R0092,8110050011,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,46
3,R0063,8110050010,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,41
4,R0113,8110020060,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,R1474,8110060012,ref,2024-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,47
119,R0089,8110060012,ref,2024-09-18,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,Yes,Yes,No,No,Female,43
120,R0393,8110050086,td,2024-08-13,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,Yes,Yes,No,No,Female,48
121,R0582,8110050094,td,2024-07-31,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,No,No,No,Male,32


In [30]:
# intake rebuild test
from src.rebuild_dataframe import *
# rebuilt_df = rebuild_dataframe(df, 'id', merge_strategies=custom_strategies, chunk_size=2)
rebuilt_intake = rebuild_dataframe(intake_dataset, 'rid')

INFO:src.rebuild_dataframe:Processed 327 records so far.


In [33]:
intake_dataset[intake_dataset.duplicated(subset=['rid'])]

,intsp,ntspi,rid,fcid,firstname,lastname,nt_s1,nt_s2,nt_s3,nt_re,...,secondary_torture,sexual_violation_a,sexual_violation_p,hrd,journalist,wov,stgbv,lgbti,other,note
167,SA,84,R0489,8110050078,مجد,محمد,2022-09-08 00:00:00,2022-09-26 00:00:00,2022-09-28 00:00:00,NaN,...,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN
193,SA,110,R0905,8110050104,علي,العمر,2023-10-25 00:00:00,2023-10-30 00:00:00,NaN,NaN,...,Yes,No,Yes,No,No,Yes,No,No,No,NaN
194,SA,111,R0864,8110050105,فدوى,حفيان,2023-10-27 00:00:00,2023-11-09 00:00:00,NaN,NaN,...,Yes,No,No,No,No,Yes,No,No,No,NaN
236,YQ,38,R0062,8110020048,سوسن,دومان,2021-05-25 00:00:00,2021-06-15 00:00:00,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN


In [31]:
rebuilt_intake

,intsp,ntspi,rid,fcid,firstname,lastname,nt_s1,nt_s2,nt_s3,nt_re,...,secondary_torture,sexual_violation_a,sexual_violation_p,hrd,journalist,wov,stgbv,lgbti,other,note
0,YQ,16,R0001,8110020026,كوثر,العيسى,2020-01-02 00:00:00,2020-01-09 00:00:00,2020-01-17 00:00:00,NaN,...,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN
1,YQ,8,R0002,8110020018,عائشة,مرندي,2020-01-03 00:00:00,2020-01-20 00:00:00,NaN,2020-10-28 00:00:00,...,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN
2,YQ,5,R0003,8110020015,عبير,خطيب,2020-01-06 00:00:00,2020-01-13 00:00:00,NaN,2021-11-23 00:00:00,...,Yes,NaN,NaN,NaN,NaN,Yes,Yes,No,No,NaN
3,YQ,9,R0004,8110020019,عبير,علوش,2020-01-06 00:00:00,2020-01-31 00:00:00,NaN,NaN,...,No,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN
4,YQ,11,R0005,8110020021,فاطمة,العلي,2020-01-06 00:00:00,2020-01-20 00:00:00,NaN,NaN,...,No,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,IJ,34,R1277,8110080027,اصالة,محمد,2024-04-25 00:00:00,2024-04-30 00:00:00,NaN,2024-09-10 00:00:00,...,Yes,No,Yes,No,No,Yes,No,No,No,NaN
323,MT,19,R1282,8110110019,منى,الخالد,2024-05-14 00:00:00,2024-05-17 00:00:00,NaN,NaN,...,Yes,No,No,No,No,Yes,No,No,No,NaN
324,MT,20,R1284,8110110020,كوثر,نزال,2024-05-16 00:00:00,2024-05-21 00:00:00,NaN,NaN,...,Yes,No,No,No,No,Yes,No,No,No,NaN
325,IJ,38,R1346,8110080029,ايمان,محمد,2024-05-23 00:00:00,2024-05-27 00:00:00,NaN,NaN,...,Yes,No,No,No,No,Yes,No,No,No,NaN


In [34]:
uniq_hsh_report_reb = pd.merge(uniq_hsh_report, rebuilt_intake[int_criteria], on='rid', how='left')
uniq_hsh_report_reb = pd.merge(uniq_hsh_report_reb, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [35]:
uniq_hsh_report_reb

,rid,fcid,service,date,family_of_missing,survivor_of_detention,primary_torture,secondary_torture,sexual_violation_a,sexual_violation_p,hrd,journalist,wov,stgbv,lgbti,other,sex,age
0,R0588,8110100010,adv,2024-08-26,Yes,No,No,Yes,No,No,No,No,Yes,No,No,No,Female,37
1,R0062,8110020048,adv,2024-08-26,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,No,No,Female,48
2,R0092,8110050011,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,46
3,R0063,8110050010,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,41
4,R0113,8110020060,adv,2024-08-26,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No,Female,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,R1474,8110060012,ref,2024-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,47
119,R0089,8110060012,ref,2024-09-18,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,Yes,Yes,No,No,Female,43
120,R0393,8110050086,td,2024-08-13,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,Yes,Yes,No,No,Female,48
121,R0582,8110050094,td,2024-07-31,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,No,No,No,Male,32
